In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score, r2_score
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import pickle
import geopandas as gpd
import numpy as np
from pykrige.rk import Krige
import matplotlib.pyplot as plt

---

In [2]:
res  =pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/pca_study/outputs/sample_c/regression_fine_tuning.csv', index_col=0)
for params, df in res.groupby(['param_max_depth', 'param_min_samples_leaf']):
    print(params, '\n', df[['mean_test_score', 'std_test_score']].mean(), '\n')

(4, 1) 
 mean_test_score    0.220119
std_test_score     0.011448
dtype: float64 

(4, 5) 
 mean_test_score    0.220064
std_test_score     0.011453
dtype: float64 

(4, 10) 
 mean_test_score    0.220088
std_test_score     0.011416
dtype: float64 

(50, 1) 
 mean_test_score    0.395458
std_test_score     0.013026
dtype: float64 

(50, 5) 
 mean_test_score    0.384570
std_test_score     0.014458
dtype: float64 

(50, 10) 
 mean_test_score    0.368164
std_test_score     0.014355
dtype: float64 

(100, 1) 
 mean_test_score    0.395458
std_test_score     0.013026
dtype: float64 

(100, 5) 
 mean_test_score    0.384570
std_test_score     0.014458
dtype: float64 

(100, 10) 
 mean_test_score    0.368164
std_test_score     0.014355
dtype: float64 



In [3]:
#### THE FINAL CHOICE IS: max_depth': 50, 'min_samples_leaf': 1

rs = 516

X_train = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/pca_study/database/sample_c/final_X_train_val.csv')
y_train = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/pca_study/database/sample_c/y_train_val.csv')

regression_forest = RandomForestRegressor(max_depth=50,
                                          min_samples_leaf=1,
                                          n_jobs=-1, 
                                          random_state=rs
                                          )


regression_forest.fit(X_train, y_train)

with open('/nfs/home/genovese/thesis-wildfire-genovese/pca_study/database/sample_c/trained_models/final_regression.pkl', 'wb') as f:
    pickle.dump(regression_forest, f)

/nfs/home/genovese/.venv/lib/python3.10/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [4]:
with open('/nfs/home/genovese/thesis-wildfire-genovese/pca_study/database/sample_c/trained_models/final_regression.pkl', 'rb') as f:
    model = pickle.load(f)

In [5]:
y_train_pred = model.predict(X_train).round(0)
print(r2_score(y_train_pred, y_train['target']))
print(accuracy_score(pd.Series(y_train_pred), pd.Series(y_train['target'])))

X_test = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/pca_study/database/sample_c/final_X_test.csv')
y_test = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/pca_study/database/sample_c/y_test.csv')

y_test_pred = model.predict(X_test).round(0)
print(r2_score(y_test_pred, y_test['target']))
print(accuracy_score(pd.Series(y_test_pred), pd.Series(y_test['target'])))

0.8528326176465565
0.41379310344827586
-0.6920404959768287
0.15794871794871795


In [6]:
def predict_dispersion(crosstab):
    total = 0
    for i, row in enumerate(crosstab.to_numpy()):
        for j, val in enumerate(row):
            total += val * abs(j - i)
    return total/(crosstab.sum().sum())*(crosstab.shape[1])

In [7]:
train_mat= pd.crosstab(y_train.target, y_train_pred)
test_mat = pd.crosstab(y_test.target, y_test_pred)

In [14]:
test_mat.loc[:, 10.0] = 0
test_mat = test_mat.loc[:, [0,1,2,3,4,5,6,7,8,9,10]]

In [15]:
print(predict_dispersion(train_mat))
print(predict_dispersion(test_mat))

7.333333333333333
20.375384615384615


---

In [16]:
res  =pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/pca_study/outputs/sample_c/classification_fine_tuning.csv', index_col=0)
for params, df in res.groupby(['param_max_depth', 'param_min_samples_leaf']):
    print(params, '\n', df[['mean_test_score', 'std_test_score']].mean(), '\n')

(4, 1) 
 mean_test_score    0.164631
std_test_score     0.011472
dtype: float64 

(4, 5) 
 mean_test_score    0.164631
std_test_score     0.009515
dtype: float64 

(4, 10) 
 mean_test_score    0.164631
std_test_score     0.010737
dtype: float64 

(50, 1) 
 mean_test_score    0.248168
std_test_score     0.008252
dtype: float64 

(50, 5) 
 mean_test_score    0.240203
std_test_score     0.009241
dtype: float64 

(50, 10) 
 mean_test_score    0.231606
std_test_score     0.010893
dtype: float64 

(100, 1) 
 mean_test_score    0.248168
std_test_score     0.008252
dtype: float64 

(100, 5) 
 mean_test_score    0.240203
std_test_score     0.009241
dtype: float64 

(100, 10) 
 mean_test_score    0.231606
std_test_score     0.010893
dtype: float64 



In [17]:
#### THE FINAL CHOICE IS: max_depth': 50, 'min_samples_leaf': 1

rs = 516

X_train = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/pca_study/database/sample_c/final_X_train_val.csv')
y_train = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/pca_study/database/sample_c/y_train_val.csv')

classification_forest = RandomForestClassifier(max_depth=50,
                                          min_samples_leaf=1,
                                          n_jobs=-1, 
                                          random_state=rs
                                          )


classification_forest.fit(X_train, y_train)

with open('/nfs/home/genovese/thesis-wildfire-genovese/pca_study/database/sample_c/trained_models/final_classification.pkl', 'wb') as f:
    pickle.dump(classification_forest, f)

/nfs/home/genovese/.venv/lib/python3.10/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [18]:
with open('/nfs/home/genovese/thesis-wildfire-genovese/pca_study/database/sample_c/trained_models/final_classification.pkl', 'rb') as f:
    model = pickle.load(f)

In [19]:
y_train_pred = model.predict(X_train).round(0)
print(r2_score(y_train_pred, y_train['target']))
print(accuracy_score(pd.Series(y_train_pred), pd.Series(y_train['target'])))

X_test = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/pca_study/database/sample_c/final_X_test.csv')
y_test = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/pca_study/database/sample_c/y_test.csv')

y_test_pred = model.predict(X_test).round(0)
print(r2_score(y_test_pred, y_test['target']))
print(accuracy_score(pd.Series(y_test_pred), pd.Series(y_test['target'])))

1.0
1.0
0.1750039918149756
0.24461538461538462


In [20]:
train_mat= pd.crosstab(y_train.target, y_train_pred)
test_mat = pd.crosstab(y_test.target, y_test_pred)

In [23]:
print(predict_dispersion(train_mat))
print(predict_dispersion(test_mat))

0.0
22.722051282051282


---